In [ ]:
# use xie_env_r.yaml/scbp_env.yaml environment 

import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import anndata as ad

plt.rcParams['figure.figsize'] = (15, 15)
from scipy.stats import median_abs_deviation

# feature selection
import anndata2ri
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

In [ ]:
# doublets filtered
#mask = (xie_filtered_doublet.obs['scDblFinder_class'] != 2) & (xie_filtered_doublet.obs['scDblFinder_score'] <= 0.9) & (xie_filtered_doublet.obs['total_counts'] <= 12000)

xie = sc.read('/home/p/pohll/Desktop/MP/data/xie_leonie_filtered.h5ad')

In [ ]:
sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

In [ ]:
sc.pl.umap(
    xie,
    color=["total_counts", "pct_counts_mt", "scDblFinder_score", "scDblFinder_class"],
)


In [ ]:
plt.rcParams['figure.figsize'] = (7, 7)
sc.pl.umap(xie, color=['annotation', 'sample'], size=30)

In [ ]:
sc.tl.leiden(xie)

In [ ]:
sc.tl.leiden(xie, key_added="leiden_res0_1", resolution=0.1)
sc.tl.leiden(xie, key_added="leiden_res0_25", resolution=0.25)
sc.tl.leiden(xie, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(xie, key_added="leiden_res1", resolution=1.0)

In [ ]:
sc.pl.umap(
    xie,
    color=["leiden_res0_1","leiden_res0_25", "leiden_res0_5", "leiden_res1"],
    legend_loc="on data",
)

## Annotation
### From markers to clusters

In [ ]:
#Define a nice colour map for gene expression
from matplotlib import colors
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,1))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
def plot_markers(adata):
    print('Macrophages')
    sc.pl.umap(adata, color=['Apoe', 'Mrc1', 'Marco', 'Mertk'], cmap=mymap, size=20)
    print('Monocytes')
    sc.pl.umap(adata, color=['Cd14', 'Vcan'], cmap=mymap, size=20)
    print('T-Cells')
    sc.pl.umap(adata, color=['Cd3e', 'Cd4','Cd8a', 'Cd3d'], cmap=mymap, size=40)
    print('B-Cells')
    sc.pl.umap(adata, color=['Cd19', 'Cd79a'], cmap=mymap, size=40)
    print('AT2')
    sc.pl.umap(adata, color=['Muc1', 'Sftpc', 'Sftpd', 'Lcn2'], cmap=mymap, size=30)
    print('AT1')
    sc.pl.umap(adata, color=['Vegfa'], cmap=mymap, size=30)
    print('Krt8')
    sc.pl.umap(adata, color='Krt8', cmap=mymap, size=30)
    print('Endothelial')
    sc.pl.umap(adata, color='Pecam1', cmap=mymap, size=30)
    print('Fibroblasts')
    sc.pl.umap(adata, color='Col1a2', cmap=mymap, size=30)
    print('Myofibroblasts')
    sc.pl.umap(adata, color=['Col3a1', 'Cthrc1', 'Postn', 'Spp1', 'Tnc', 'S100a6', 'Ccl2'], cmap=mymap, size=30)
    
def plot_marker_genes(adata):
    plot_markers(adata)

In [ ]:
plot_markers(xie)

In [ ]:
xie.obs['cond']='control'

In [ ]:
xie.obs['sample']=xie.obs.index.str.split('-').str[1]

In [ ]:
xie.obs.loc[xie.obs['sample'].isin(['4', '5', '6']), 'cond']='bleo'

In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)
sc.pl.umap(xie, color=['sample', 'cond'], size=30)

### From cluster differentially expressed genes to cluster annotation

In [ ]:
sc.pl.umap(xie, color="leiden_res0_5")

In [ ]:
# todo manual annotation? 

In [ ]:
#xie.obs["manual_celltype_annotation"] = xie.obs.leiden_2.map(cl_annotation)

In [ ]:
xie.uns['log1p'] = {'base': None}
# https://github.com/scverse/scanpy/issues/2181

In [ ]:
sc.tl.rank_genes_groups(
    xie, groupby="leiden_res0_5", method="wilcoxon", key_added="dea_leiden_0.5"
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    xie, groupby="leiden_res0_5", standard_scale="var", n_genes=5, key="dea_leiden_0.5"
)

In [ ]:
sc.tl.filter_rank_genes_groups(
    xie,
    min_in_group_fraction=0.2,
    max_out_group_fraction=0.2,
    key="dea_leiden_0.5",
    key_added="dea_leiden_0_5_filtered",
)



In [ ]:
sc.pl.rank_genes_groups_dotplot(
    xie,
    groupby="leiden_res0_5",
    standard_scale="var",
    n_genes=5,
    key="dea_leiden_0_5_filtered",
)



todo manual celltype annotation 

Automated annotation --> cellltypist not mouse lung model 